## Demo 2 - Persistent homology computations and interpretation

In [ ]:
!pip install ripser

In [ ]:
import time
import numpy as np

from sklearn import datasets

#topological data analysis
from ripser import ripser
from persim import plot_diagrams

#plotting and visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.spatial import distance

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_barcodes(diagrams, alpha_max, width = 1.5):
    max_dim = len(diagrams)
    fig, axs = plt.subplots(max_dim)
    fig.suptitle('Barcodes')
    for dim in range(max_dim):
        barcode = np.copy(diagrams[dim])
        ind_inf = np.isinf(barcode.T[1])
        barcode[ind_inf, 1] = alpha_max
        h = 1
        for i in range(len(barcode)):
            x = barcode[i]
            y = [h,h]
            axs[dim].plot(x, y, linestyle= '-', c='#1f77b4', linewidth = width)
            if ind_inf[i]:
                axs[dim].scatter([alpha_max],[h],  s=10, marker='>', c='#1f77b4')
            h += 1
        axs[dim].set_xlim(0, 1.05*alpha_max)
        axs[dim].set_ylim(0,h)
        axs[dim].get_yaxis().set_ticks([]);
        axs[dim].spines['right'].set_color('none')
        axs[dim].spines['top'].set_color('none')
        axs[dim].text(0.3,1,'$\mathrm{bcd}^{\mathcal{R}}_{'+str(dim)+'}(X)$', verticalalignment='bottom')

### Rips Filtration

Given a dataset $(X,\mathbf{d}_X)$, its Rips filtration is defined as:

$$\mathcal{R}(X) = \Big\{\, i_{\alpha',\alpha}: R_\alpha(X) \hookrightarrow R_{\alpha'} (X)\, \Big\}_{0 < \alpha\leq \alpha'} $$
where
$$R_\alpha(X) = \Big\{\, \sigma \subset X \mid 0 < \#(\sigma) < \infty \;\mbox{ and }\; \mathsf{diam}(\sigma) < \alpha\, \Big\} $$

**Do:** Use the cell below to visualize the Rips complex of the provided dataset at various distance thresholds.

In [ ]:
np.random.seed(1223)

n_data = 30
theta = np.random.uniform(0, 2*np.pi, n_data)
data = np.array([np.cos(theta) , np.sin(theta) , np.zeros_like(theta)])
data += np.random.normal(0, 0.08, data.shape)

fig = go.Figure(data=[go.Scatter3d(
    x=data[0], y=data[1], z=data[2],
    mode ='markers',
    marker=dict(size = 3 , color = 'grey'))])

fig.update_layout(scene= dict(zaxis = dict(range=[-1, 1])))
fig.show()

In [ ]:
alpha = .1
distMat = distance.squareform(distance.pdist(data.T))

ii = []; jj = []; kk = []
e_x =[]; e_y =[]; e_z =[]

for i in range(n_data):
    for j in range(i+1,n_data):
        if distMat[i,j] < alpha:
            # add edge (i,j)
            e_x.extend([data[0,i], data[0,j], None])
            e_y.extend([data[1,i], data[1,j], None])
            e_z.extend([data[2,i], data[2,j], None])

            for k in range(j+1,n_data):
                if np.max([distMat[j,k], distMat[i,k]]) < alpha:
                    # add triangle (i,j,k)
                    ii.append(i); jj.append(j); kk.append(k)

vertices = go.Scatter3d(mode = 'markers', name = 'vertices',
                        x = data[0], y = data[1],  z = data[2],
                        marker=dict(size = 3 , color = 'grey'))

edges = go.Scatter3d(mode='lines', name = 'edges',
                     x=e_x, y=e_y, z=e_z,
                     line=dict(color= 'rgb(70,70,70)', width=1))

triangles = go.Mesh3d(x=data[0], y=data[1], z=data[2],  i = ii, j = jj, k = kk,  color='lightpink', opacity=0.2)

fig = go.Figure(data=[vertices, edges, triangles])
fig.update_traces(hoverinfo="none")
fig.update_layout(scene= dict(
                      xaxis = dict(showspikes=False),
                      yaxis = dict(showspikes=False),
                      zaxis = dict(showspikes=False,range=[-1, 1])))
fig.show()

### The Matrix Reduction Algorithm

The computation of persistence diagrams/barcodes takes as input a simplexwise filtration of a simplicial complex $K$:
$$K_0 \subset K_1 \subset \cdots \subset K_N  = K $$
where $K_{0} = \emptyset$ and $K_{n} = K_{n-1} \cup \{\sigma_n\}$ for some simplex $\sigma_n \in K\smallsetminus K_{n-1}$ and  $n =  1, \ldots, N$. The $N\times N$ *total boundary matrix* $D$ of the filtration has entries $D[i,j] = \pm 1$ if $\sigma_i $ is a condimension 1 face of $\sigma_j$  (the sign is determined by the orientation), and $D[i,j] = 0$ otherwise. The Matrix Reduction Algorithm for computing persistence can be implemented as follows:
<pre>
<b>Input:</b>  Sorted total boundary matrix D (size NxN)
<b>Output:</b> Reduced matrix R
        List of persistence pairs P
        Invertible upper triangular  matrix V  (R = D*V)

P = []
V = I
R = D

<b>for</b> j = 1 to N <b>do</b>
    <b>while</b> there exists k < j with Pivot(R_k) = Pivot(R_j) <b>do</b>
        c = PivotEntry(R_j)/PivotEntry(R_k)
        R_j := R_j - c*R_k
        V_j := V_j - c*V_k

     <b>if</b> (i = Pivot(R_j)) != 0 <b>then</b>
         append(i,j) to P
     <b>else</b>
         append (j, infty) to P

<b>return</b> V,R,P
</pre>

The paper introducing this algorithm is https://geometry.stanford.edu/papers/zc-cph-05/zc-cph-05.pdf

### Computing Persistence via Ripser

U. Bauer: "Ripser is a lean C++ code for the computation of Vietoris–Rips persistence barcodes. It can do just this one thing, but does it extremely well."

Original C++ library : https://github.com/Ripser/ripser

Accompanying paper : https://arxiv.org/pdf/1908.02518.pdf

Python library: https://ripser.scikit-tda.org/en/latest


In [ ]:
# Persistence Computation
start_time = time.time()
rips_persistence = ripser(data.T, maxdim=1)
print("--- %s seconds ---" % (time.time() - start_time))

dgms = rips_persistence['dgms']
plot_barcodes(dgms,1.8);

In [ ]:
plt.figure(figsize = (3,3))
plot_diagrams(dgms, title='Persistence Diagrams')

### Data Example: Blobs


In [ ]:


plt.figure(figsize = (3,3))
plt.scatter(data.T[0], data.T[1], s = 3, c = y_true, marker= '');
plt.axis('square');

In [ ]:
n_data = 50
n_clusters = 3

data, y_true = datasets.make_blobs(n_samples=n_data, centers=n_clusters, cluster_std=0.30, random_state= 0)

fig = px.scatter(x=data.T[0], y=data.T[1], color = y_true.astype(str))
fig.update_layout(width=500, height=500)
fig.show()

In [ ]:
# Complte the cell below

max_distance =
max_homology =

rips_persistence = ripser(data, maxdim= max_homology, thresh = max_distance)

dgms = rips_persistence['dgms']
plot_barcodes(dgms, alpha_max = max_distance, width = 1);

#plt.figure(figsize = (3,3))
#plot_diagrams(dgms, title='Persistence Diagrams')

In [ ]:
plt.figure(figsize = (3,3))
plot_diagrams(dgms, title='Persistence Diagrams')

### Data Example: Trefoil Knot

In [ ]:
# Setup trefoil knot data
np.random.seed(1122)
n_data = 5000

u = 4*np.pi*np.random.rand(n_data)
v = 2*np.pi*np.random.rand(n_data)
data = np.zeros((3,n_data))

data[0] = np.cos(u)*np.cos(v) + 6*np.cos(u)*(1.5+np.sin(1.5*u)/2)
data[1] = np.sin(u)*np.cos(v) + 6*np.sin(u)*(1.5+np.sin(1.5*u)/2)
data[2] = np.sin(v) + 4*np.cos(1.5*u)
data += 0.2*np.random.randn(*data.shape)

# Plot the data
fig = go.Figure(data=[go.Scatter3d(
    x=data[0], y=data[1], z=data[2],
    mode ='markers',
    marker=dict(size = 1.5 , color = 'grey'))])

fig.update_layout( width=900, height=450)
fig.show()

In [ ]:
# Persistence Computation
start_time = time.time()
n_landmarks = 200
prime_coeff = 7

rips_persistence = ripser(data.T, n_perm = n_landmarks, coeff = prime_coeff, maxdim=1)
print("--- %s seconds ---" % (time.time() - start_time))
dgms = rips_persistence['dgms']
idx_perm = rips_persistence['idx_perm']
plt.figure(figsize = (3,3))
plot_diagrams(dgms)

In [ ]:
plot_barcodes(dgms, 10, 1)

**Note:** When there are too many intervals, the barcode is not that useful a visualization.

### Subsampling the data

If the dataset $(X,\mathbf{d}_X)$  is too large -- that is, if $N = \#(X)>>1$ -- then the persistence computation can be very slow. There are two strategies to get around this (the stability theorem -- we'll talk about this soon -- implies that these are good ideas):

#### Uniform  subsample:

Having indexed the points of $X$ arbitrarily as $X = \{x_1, \ldots, x_N\}$, choose indices $\{n_1, \ldots , n_k\} \subset \{1,\ldots, N\}$ uniformly at random (without replacement). Let $\tilde{X} = \{x_{n_1}, \ldots, x_{n_k}\} \subset X$ be the subsample.


#### MaxMin sampling:

Choose $x_1 \in X$ uniformly at random, and given $x_1,\ldots, x_{j} \in X$, let
$$
x_{j+1} = \underset{x \in X}{\mathsf{argmax}} \,  \mathsf{min} \Big\{ \mathbf{d}_X(x_1, x) ,\ldots, \mathbf{d}_X(x_j, x)\Big\}
$$



In [ ]:
# The data
original_cloud =  go.Scatter3d( x=data[0], y=data[1], z=data[2],
                               mode ='markers', name = 'data', marker=dict(size = 1.5 , color = 'grey'))

# The uniform sample
np.random.seed(1122)
idx_uni = np.random.choice(np.arange(0,data.shape[1]), len(idx_perm))
uniform = go.Scatter3d( x=data[0,idx_uni], y=data[1,idx_uni], z=data[2,idx_uni],
                               mode ='markers', name = 'uniform', marker=dict(size = 2 , color = 'blue'))

# The maxim sample
maxmin =  go.Scatter3d( x=data[0,idx_perm], y=data[1,idx_perm], z=data[2,idx_perm],
                               mode ='markers', name = 'maxmin', marker=dict(size = 2 , color = 'red'))

fig = go.Figure(data=[original_cloud, uniform, maxmin])

fig.update_layout( width=900, height=450)
fig.show()

### Data Example: Torus

In [ ]:
np.random.seed(2)
n_data = 25000
R = 5
r = 2
data = np.zeros((3, n_data))
s = np.random.rand(n_data)*2*np.pi
t = np.random.rand(n_data)*2*np.pi

data[0] = (R + r*np.cos(s))*np.cos(t)
data[1] = (R + r*np.cos(s))*np.sin(t)
data[2] = r*np.sin(s)
data += 0.1*np.random.randn(*data.shape)

# Plot the data
fig = go.Figure(data=[go.Scatter3d(
    x=data[0], y=data[1], z=data[2],
    mode ='markers',
    marker=dict(size = 1.5 , color = 'grey'))])

fig.update_layout( width=900, height=450)
fig.show()

In [ ]:
# Persistence Computation





## Activity:

1. Load the mistery data sets
2. Compute their persistent homology
3. Given the results, what topoloical spaces could they be sampling?

In [ ]:
# Your code here


